In [1]:
import pandas as pd
import json
import requests
from io import StringIO
from csv import reader
import os
import glob
import numpy as np
import shutil
import time
import datetime

In [2]:
class Searchfilter():
    def __init__(self, file_url, all_ids_file="all ids we have.csv"):
        self.file_url = file_url
        self.all_ids_file = all_ids_file

        # read search result:
        print("\n", self.file_url.split('/')[2])
        result = pd.read_csv(file_url)
        print("All results:", result.shape[0])

        # drop results without IDs
        result = result.dropna(subset=['id'])

        # set all letters to be lower case to search on "Amber word"
        result['title'] = result['title'].str.lower()
        result['description'] = result['description'].str.lower()

        # search for the words in titles
        mask = np.column_stack([result['title'].str.contains(r"amber", na=False) for col in result])
        result_title = list(result.loc[mask.any(axis=1)]['id'])

        # search for the words in descriptons
        mask = np.column_stack([result['description'].str.contains(r"amber", na=False) for col in result])
        result_description = list(result.loc[mask.any(axis=1)]['id'])

        result = result_description + result_title
        result = set(result)
        result = list(result)
        print("results with word 'Amber' in title or description:", len(result))

        # read IDs we already scraped:
        all_ids = pd.read_csv(all_ids_file)
        all_ids = list(all_ids.id)

        # filter the IDs of new search with the IDs we have
        uniqu_ids = []
        for i in result:
            if i not in all_ids:
                uniqu_ids.append(i)
        print("Unique Id in {} search:".format(self.file_url.split('/')[2]), len(uniqu_ids))

        # save unique IDs to the same file we read
        uniqu_ids_df = pd.DataFrame(uniqu_ids, columns=['id'])
        uniqu_ids_df.to_csv(file_url, index=None)

        # add the new results to the all IDs we have:
        print("all ids before adding:", len(all_ids))
        all_ids = all_ids + uniqu_ids
        all_ids = set(all_ids)
        all_ids = list(all_ids)
        print("all ids after adding:", len(all_ids))
        all_ids_df = pd.DataFrame(all_ids, columns=['id'])
        all_ids_df.to_csv(all_ids_file, index=None)
        print('File saved ..')
        id_count = len(uniqu_ids)
        

In [3]:
# read josn files and make a list with json files

path_to_csv = "./need clean"
csv_files = [pos_csv for pos_csv in os.listdir(path_to_csv) if pos_csv.endswith('.csv')]
len(csv_files)

31

In [4]:
for file in csv_files:
    Searchfilter(path_to_csv+"/"+file)   


 401- Amber Heard Playlist - Sheet1.csv
All results: 581
results with word 'Amber' in title or description: 433
Unique Id in 401- Amber Heard Playlist - Sheet1.csv search: 72
all ids before adding: 20184
all ids after adding: 20256
File saved ..

 402- Amber Heard Car - Sheet1.csv
All results: 712
results with word 'Amber' in title or description: 509
Unique Id in 402- Amber Heard Car - Sheet1.csv search: 30
all ids before adding: 20256
all ids after adding: 20286
File saved ..

 403- Amber Heard Top Gear - Sheet1.csv
All results: 715
results with word 'Amber' in title or description: 337
Unique Id in 403- Amber Heard Top Gear - Sheet1.csv search: 23
all ids before adding: 20286
all ids after adding: 20309
File saved ..

 404- Amber Heard Best - Sheet1.csv
All results: 688
results with word 'Amber' in title or description: 539
Unique Id in 404- Amber Heard Best - Sheet1.csv search: 13
all ids before adding: 20309
all ids after adding: 20322
File saved ..

 405- Amber Heard Tribute - S